# Task 1
 


Write a class object that contains the different numerical approaches to integration. Compare accuracy and speed of the different approaches. Which approach works well for which function? Which functions are difficult to integrate with a Monte Carlo approach?

Make sure to validate your results using analytical integrals. 

# Task 2

Line out how you would proceed to implement 2D integrals for functions $f_5$ and $f_6$ using a 2D Riemann method (no need to implement, we will do something similar later) and a Monte-Carlo integral. 

Make a statement about the scaling of the number of numerical operations compared to 1D integrals in case of the Riemann method. Does this scaling also apply to the Monte-Carlo method?


# Optional: 
Argue if the structure of functions $f_5$ and $f_6$ can be exploited to implement the Riemann integrals in a more efficient way than one would use for arbitrary 2D functions. Hint: $\sin(x+y) = \sin(x) \cdot \cos(y) + \cos(x) \cdot \sin(y)$


Upload your notebook to moodle.

In [1]:
from numpy import *
import time
from tabulate import tabulate

In [2]:
def function1(x):
    y = x*(x - 3)*(x + 3)
    return y

def function2(x):
    y = abs(x)
    return y

def function3(x):
    y = sin(x * 2.1) * (-x / 2.0)
    return y

def function4(x):
    y = 1.6 ** x - 1.5 * x
    return y

#################################################################


def integral1(a,b):
    # integral of function1 over [a,b]
    return (b**4/4 - 9*b**2/2) - (a**4/4 - 9*a**2/2)

def integral2(a,b):
    # integral of function2 over [a,b]
    a2 = a**2/2*sign(a)
    b2 = b**2/2*sign(b)
    return b2 - a2

def integral3(a,b):
    # integral of function3 over [a,b]
    uv = -cos(b*2.1)/2.1 * (-b / 2.0) + cos(a*2.1)/2.1 * (-a / 2.0)
    uvp = -sin(b*2.1)/2.1**2 /2 + sin(a*2.1)/2.1**2 /2 
    return uv + uvp

def integral4(a,b):
    # integral of function4 over [a,b]
    lg = 1.6**b/log(1.6) - 1.6**a/log(1.6)
    sq = 1.5 * b**2/2 - 1.5 * a**2/2
    return lg - sq


## Task 1

Write a class object that contains the different numerical approaches to integration. Compare accuracy and speed of the different approaches. Which approach works well for which function? Which functions are difficult to integrate with a Monte Carlo approach?

Make sure to validate your results using analytical integrals. 

In [3]:
class integrate:
    def __init__(self, func, ana, N, a, b):
        self.a = a
        self.b = b
        self.N = N
        self.x, self.dx = linspace(a, b, N, endpoint=True, retstep=True)
        self.f = func
        self.y = func(self.x)
        self.a = ana(a, b)
        
    def getTime(self):
        return time.process_time()
    
    def riemann(self):
        t0 = self.getTime()
        
        ff = self.f(self.x)
        I =ff.sum()*self.dx
        t1 = self.getTime()
        
        err=self.a - I
        
        return I, t1-t0, err
        
    def midpoint(self):
        t0 = self.getTime()
        
        ff = self.f(self.x+self.dx/2)
        I = ff.sum()*self.dx
        t1 = self.getTime()
        
        err=self.a - I
        
        return I, t1-t0, err
    
    def trapezoid(self):
        t0 = self.getTime()
        
        if self.N == 1:
            I = (self.b-self.a)/0.5*(self.x[0]+self.x[-1])
            t1 = self.getTime()
            err=self.a - I
            return I, t1-t0
        else:
            I = (self.y[1:self.N-1].sum() + (self.y[0] + self.y[-1])/2)*self.dx
            t1 = self.getTime()
            err=self.a - I
            return I, t1-t0, err
        
    def montecarlo(self): #integrate_MC1D
        fmax = self.y.max()
        fmin = self.y.min()

        # just to avoid zero volume
        if abs(fmax-fmin) < 1.e-8:
            fmin = fmin - 1.e-8
            fmax = fmax + 1.e-8

        t0 = self.getTime()
            
        x_rand = (self.b-self.a)*random.random(self.N)+self.a
        y_rand = (fmax-fmin)*random.random(self.N)+fmin

        # evaluate this only once
        func_rand = self.f(x_rand)


        ind_below_pos = nonzero((y_rand <= func_rand) & (y_rand >= 0))
        ind_below_neg = nonzero((y_rand >= func_rand) & (y_rand < 0))
        ind_above_pos = nonzero((y_rand > func_rand) & (y_rand >= 0))
        ind_above_neg = nonzero((y_rand < func_rand) & (y_rand < 0))

        integral = (len(ind_below_pos[0])-len(ind_below_neg[0]))/self.N
        #print('Ratio of sampled points within integration are vs total sample are', sampled)

        area = (fmax-fmin)*(self.b-self.a)
        I = area*integral
        
        t1 = self.getTime()
        
        npos = len(ind_above_pos[0])
        nneg = len(ind_below_neg[0])

        meanfsquare = ((npos - nneg)/N)**2
        fsquaremean = (npos + nneg)/N  # f^2 is one for all points that count, zero otherwise

        err = area*sqrt((fsquaremean - meanfsquare)/N)
                
        return I, t1-t0, err

    def sampling(self): #integrate_MC1D_direct
        x_rand = (self.b-self.a)*random.random(self.N)+self.a
        y_rand = self.f(x_rand)
        
        t0 = self.getTime()

        I = sum(y_rand)*(self.b-self.a)/self.N
        #print('Monte Carlo integral:', integral)

        # error estimate:

        t1 = self.getTime()
        
        err = sqrt(((y_rand**2).mean() - y_rand.mean()**2)/self.N)*(self.b-self.a)
        #print('Estimated Monte-Carlo error:', error)

        return I, t1-t0, err

In [4]:
N = 20000
a = -4
b = 4
funcs=[1, 2, 3, 4, 5]

Function 1 - Riemann

Function 2 - Riemann midpoint

Function 3 - trapezoid

Function 4 - MC Hit Miss

Function 5 - MC Sampling

In [5]:
f1 = integrate(function1, integral1, N, a, b)

I1 = zeros(5)
dt1 = zeros(5)
err1 = zeros(5)

I1[0], dt1[0], err1[0] = f1.riemann()
I1[1], dt1[1], err1[1] = f1.midpoint()
I1[2], dt1[2], err1[2] = f1.trapezoid()
I1[3], dt1[3], err1[3] = f1.montecarlo()
I1[4], dt1[4], err1[4] = f1.sampling()

data1 = zeros(20)
data1 = data1.reshape(5, 4)

#funcs=['Riemann', 'Riemann midpoint', 'trapezoid', 'MC Hit Miss', 'MC Sampling']

for i in range(5):
    data1[i] = [funcs[i], I1[i], dt1[i], err1[i]]
    
myheaders = ['functions', 'Integral', 'time needed', 'error']
table1 = tabulate(data1, headers=myheaders, tablefmt='fancy_grid')
print(table1)

╒═════════════╤══════════════╤═══════════════╤══════════════╕
│   functions │     Integral │   time needed │        error │
╞═════════════╪══════════════╪═══════════════╪══════════════╡
│           1 │ -1.30974e-14 │   0.0011374   │  1.30974e-14 │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           2 │  0.0112037   │   0.000108484 │ -0.0112037   │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           3 │ -4.36579e-15 │   2.2672e-05  │  4.36579e-15 │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           4 │ -7.448       │   0.00103659  │  1.00905     │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           5 │ -8.15978     │   0.000148078 │  0.270062    │
╘═════════════╧══════════════╧═══════════════╧══════════════╛


In [6]:
f2 = integrate(function2, integral2, N, a, b)

I2 = zeros(5)
dt2 = zeros(5)
err2 = zeros(5)

I2[0], dt2[0], err2[0] = f1.riemann()
I2[1], dt2[1], err2[1] = f1.midpoint()
I2[2], dt2[2], err2[2] = f1.trapezoid()
I2[3], dt2[3], err2[3] = f1.montecarlo()
I2[4], dt2[4], err2[4] = f1.sampling()

data2 = zeros(20)
data2 = data1.reshape(5, 4)

for i in range(5):
    data2[i] = [funcs[i], I2[i], dt2[i], err2[i]]
    
myheaders = ['functions', 'Integral', 'time needed', 'error']
table2 = tabulate(data2, headers=myheaders, tablefmt='fancy_grid')
print(table2)

╒═════════════╤══════════════╤═══════════════╤══════════════╕
│   functions │     Integral │   time needed │        error │
╞═════════════╪══════════════╪═══════════════╪══════════════╡
│           1 │ -1.30974e-14 │    0.00293145 │  1.30974e-14 │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           2 │  0.0112037   │    0.0017123  │ -0.0112037   │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           3 │ -4.36579e-15 │    2.9185e-05 │  4.36579e-15 │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           4 │ -8.7136      │    0.00104566 │  1.01597     │
├─────────────┼──────────────┼───────────────┼──────────────┤
│           5 │ -8.17028     │    7.0823e-05 │  0.26882     │
╘═════════════╧══════════════╧═══════════════╧══════════════╛


In [7]:
f3 = integrate(function3, integral3, N, a, b)

I3 = zeros(5)
dt3 = zeros(5)
err3 = zeros(5)

I3[0], dt3[0], err3[0] = f3.riemann()
I3[1], dt3[1], err3[1] = f3.midpoint()
I3[2], dt3[2], err3[2] = f3.trapezoid()
I3[3], dt3[3], err3[3] = f3.montecarlo()
I3[4], dt3[4], err3[4] = f3.sampling()

data3 = zeros(20)
data3 = data1.reshape(5, 4)

for i in range(5):
    data3[i] = [funcs[i], I3[i], dt3[i], err3[i]]
    
myheaders = ['functions', 'Integral', 'time needed', 'error']
table3 = tabulate(data3, headers=myheaders, tablefmt='fancy_grid')
print(table3)

╒═════════════╤════════════╤═══════════════╤══════════════╕
│   functions │   Integral │   time needed │        error │
╞═════════════╪════════════╪═══════════════╪══════════════╡
│           1 │  -1.18359  │   0.000526738 │  0.000683667 │
├─────────────┼────────────┼───────────────┼──────────────┤
│           2 │  -1.18359  │   0.000741522 │  0.000683596 │
├─────────────┼────────────┼───────────────┼──────────────┤
│           3 │  -1.18291  │   7.1074e-05  │ -4.67704e-08 │
├─────────────┼────────────┼───────────────┼──────────────┤
│           4 │  -0.875082 │   0.00209958  │  0.0704825   │
├─────────────┼────────────┼───────────────┼──────────────┤
│           5 │  -0.925128 │   6.6766e-05  │  0.0284428   │
╘═════════════╧════════════╧═══════════════╧══════════════╛


In [8]:
f4 = integrate(function4, integral4, N, a, b)

I4 = zeros(5)
dt4 = zeros(5)
err4 = zeros(5)

I4[0], dt4[0], err4[0] = f4.riemann()
I4[1], dt4[1], err4[1] = f4.midpoint()
I4[2], dt4[2], err4[2] = f4.trapezoid()
I4[3], dt4[3], err4[3] = f4.montecarlo()
I4[4], dt4[4], err4[4] = f4.sampling()

data4 = zeros(20)
data4 = data1.reshape(5, 4)

for i in range(5):
    data4[i] = [funcs[i], I3[i], dt3[i], err3[i]]
    
myheaders = ['functions', 'Integral', 'time needed', 'error']
table4 = tabulate(data4, headers=myheaders, tablefmt='fancy_grid')
print(table4)

╒═════════════╤════════════╤═══════════════╤══════════════╕
│   functions │   Integral │   time needed │        error │
╞═════════════╪════════════╪═══════════════╪══════════════╡
│           1 │  -1.18359  │   0.000526738 │  0.000683667 │
├─────────────┼────────────┼───────────────┼──────────────┤
│           2 │  -1.18359  │   0.000741522 │  0.000683596 │
├─────────────┼────────────┼───────────────┼──────────────┤
│           3 │  -1.18291  │   7.1074e-05  │ -4.67704e-08 │
├─────────────┼────────────┼───────────────┼──────────────┤
│           4 │  -0.875082 │   0.00209958  │  0.0704825   │
├─────────────┼────────────┼───────────────┼──────────────┤
│           5 │  -0.925128 │   6.6766e-05  │  0.0284428   │
╘═════════════╧════════════╧═══════════════╧══════════════╛


In [9]:
data5 = [
    [I1[0], dt1[0], err1[0]],
    [I2[0], dt2[0], err2[0]],
    [I3[0], dt3[0], err3[0]],
    [I4[0], dt4[0], err4[0]],
]

myheaders = ['Integral', 'time needed', 'error']
table5 = tabulate(data5, headers=myheaders, tablefmt='fancy_grid')

print('comparing Riemann function:')
print(table5)

comparing Riemann function:
╒══════════════╤═══════════════╤══════════════╕
│     Integral │   time needed │        error │
╞══════════════╪═══════════════╪══════════════╡
│ -1.30974e-14 │   0.0011374   │  1.30974e-14 │
├──────────────┼───────────────┼──────────────┤
│ -1.30974e-14 │   0.00293145  │  1.30974e-14 │
├──────────────┼───────────────┼──────────────┤
│ -1.18359     │   0.000526738 │  0.000683667 │
├──────────────┼───────────────┼──────────────┤
│ 13.6204      │   0.00178941  │ -0.00134134  │
╘══════════════╧═══════════════╧══════════════╛


In [10]:
data6 = [
    [I1[1], dt1[1], err1[1]],
    [I2[1], dt2[1], err2[1]],
    [I3[1], dt3[1], err3[1]],
    [I4[1], dt4[1], err4[1]],
]

myheaders = ['Integral', 'time needed', 'error']
table6 = tabulate(data6, headers=myheaders, tablefmt='fancy_grid')

print('comparing Riemann midpoint function:')
print(table6)

comparing Riemann midpoint function:
╒════════════╤═══════════════╤══════════════╕
│   Integral │   time needed │        error │
╞════════════╪═══════════════╪══════════════╡
│  0.0112037 │   0.000108484 │ -0.0112037   │
├────────────┼───────────────┼──────────────┤
│  0.0112037 │   0.0017123   │ -0.0112037   │
├────────────┼───────────────┼──────────────┤
│ -1.18359   │   0.000741522 │  0.000683596 │
├────────────┼───────────────┼──────────────┤
│ 13.6193    │   0.00100607  │ -0.000221557 │
╘════════════╧═══════════════╧══════════════╛


In [11]:
data7 = [
    [I1[2], dt1[2], err1[2]],
    [I2[2], dt2[2], err2[2]],
    [I3[2], dt3[2], err3[2]],
    [I4[2], dt4[2], err4[2]],
]

myheaders = ['Integral', 'time needed', 'error']
table7 = tabulate(data7, headers=myheaders, tablefmt='fancy_grid')

print('comparing trapezoid function:')
print(table7)

comparing trapezoid function:
╒══════════════╤═══════════════╤══════════════╕
│     Integral │   time needed │        error │
╞══════════════╪═══════════════╪══════════════╡
│ -4.36579e-15 │    2.2672e-05 │  4.36579e-15 │
├──────────────┼───────────────┼──────────────┤
│ -4.36579e-15 │    2.9185e-05 │  4.36579e-15 │
├──────────────┼───────────────┼──────────────┤
│ -1.18291     │    7.1074e-05 │ -4.67704e-08 │
├──────────────┼───────────────┼──────────────┤
│ 13.6191      │    5.0064e-05 │ -4.01173e-08 │
╘══════════════╧═══════════════╧══════════════╛


In [12]:
data8 = [
    [I1[3], dt1[3], err1[3]],
    [I2[3], dt2[3], err2[3]],
    [I3[3], dt3[3], err3[3]],
    [I4[3], dt4[3], err4[3]],
]

myheaders = ['Integral', 'time needed', 'error']
table8 = tabulate(data8, headers=myheaders, tablefmt='fancy_grid')

print('comparing MC hit miss function:')
print(table8)

comparing MC hit miss function:
╒════════════╤═══════════════╤════════════╕
│   Integral │   time needed │      error │
╞════════════╪═══════════════╪════════════╡
│  -7.448    │    0.00103659 │  1.00905   │
├────────────┼───────────────┼────────────┤
│  -8.7136   │    0.00104566 │  1.01597   │
├────────────┼───────────────┼────────────┤
│  -0.875082 │    0.00209958 │  0.0704825 │
├────────────┼───────────────┼────────────┤
│ -53.2904   │    0.00155218 │ -0.13207   │
╘════════════╧═══════════════╧════════════╛


In [13]:
data9 = [
    [I1[4], dt1[4], err1[4]],
    [I2[4], dt2[4], err2[4]],
    [I3[4], dt3[4], err3[4]],
    [I4[4], dt4[4], err4[4]],
]

myheaders = ['Integral', 'time needed', 'error']
table9 = tabulate(data9, headers=myheaders, tablefmt='fancy_grid')

print('comparing MC sampling function:')
print(table9)

comparing MC sampling function:
╒══════════════╤═══════════════╤════════════╕
│     Integral │   time needed │      error │
╞══════════════╪═══════════════╪════════════╡
│    -8.15978  │   0.000148078 │  0.270062  │
├──────────────┼───────────────┼────────────┤
│    -8.17028  │   7.0823e-05  │  0.26882   │
├──────────────┼───────────────┼────────────┤
│    -0.925128 │   6.6766e-05  │  0.0284428 │
├──────────────┼───────────────┼────────────┤
│ -1115.17     │   7.2776e-05  │ -9.87973   │
╘══════════════╧═══════════════╧════════════╛



## Task 2

Line out how you would proceed to implement 2D integrals for functions $f_5$ and $f_6$ using a 2D Riemann method (no need to implement, we will do something similar later) and a Monte-Carlo integral. 

Make a statement about the scaling of the number of numerical operations compared to 1D integrals in case of the Riemann method. Does this scaling also apply to the Monte-Carlo method?

---

It is quite similar to the 1D case. This time, we evaluate the volume between the 2D function and the xy-plane. This means, that we take the sum over cuboids instead of rectangles.

Needing to evaluate along the x- as well as y-axis means we will have $n^2$ cuboids compared to $n$ rectangles. This increases exponentially with the amount of dimensions we have to integrate over.

For Monte-Carlo, since we evaluate an amount of random points, it doesn't really matter how many dimensions are being considered, since they will be (more or less) distributed evenly around a certain area (1D) or space (2D). With increasing dimensions, the Monte-Carlo approaches should scale linear compared to the exponential scaling of the Riemann approach. This would make the Monte-Carlo approach more viable for higher dimensions.

I would imagine there to be a difference in accuracy between 1D and 2D functions with the same amount of points. Mainly, because the average distance between the points would increase from $\sqrt{2 \cdot a^2}$ to $\sqrt{3 \cdot a^2}$. This assumes an average distance along the x and y axis in the 1D functions, denoted as $a$. The average distance between two random points is then the diagonal of a triangle ($\sqrt{2 \cdot a^2}$). For 2D, we would then have a cube and the diagonal would be ($\sqrt{3 \cdot a^2}$).

We would then need an increase of $\sqrt{\frac{3}{2}}$ points to have the same amount of "quality" or "accuracy" of the numerical integration (this is a guess, not entirely sure, if that makes sense).

## Optional: 
Argue if the structure of functions $f_5$ and $f_6$ can be exploited to implement the Riemann integrals in a more efficient way than one would use for arbitrary 2D functions. Hint: $\sin(x+y) = \sin(x) \cdot \cos(y) + \cos(x) \cdot \sin(y)$

\begin{align}
    f_5(x,y) &= \sin\left(x+y\right)\tan\left(0.1x\right) \\
    f_6(x,y) &= \sin\left(\sqrt{5}+x\right)y 
\end{align}

Function 5: 